# Kredi Kartı Harcama Analizi

In [ ]:
# gerekli kutuphaneler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter("ignore")

In [ ]:
users = pd.read_csv("users.csv")
transactions = pd.read_csv("transactions.csv")

### Veri Setlerinin İncelenmesi

In [ ]:
users.head()

In [ ]:
users.info()

Users veri setindeki bazı sütunlarının (_Per Capital Income - Zip Code_, _Yearly Income - Person_ ve _Total Debt_) başında `$` işareti bulunuyor. Bu dolar işaretlerini kaldırıp daha sonrasında bu sütunların tipini numeric (`float`) yapmalıyız, bu sayede daha kolay analiz yapabiliriz.

In [ ]:
for col in ["Per Capita Income - Zipcode", "Yearly Income - Person", "Total Debt"]:
	users[col] = users[col].astype(str).str.replace('$', '', regex= False).astype(float)

In [ ]:
users.head()

_Apartment_ sütunu çok fazla `NaN` değere sahip. Bu şekilde bırakmak veri setinin tutarsız olmasına sebep olur, bu sebeple boş değerleri `None` yapmalıyız.

In [ ]:
users["Apartment"] = users["Apartment"].fillna("None").astype(str)

_Zipcode_ sütunu tanımlayıcıdır ve bu sebeple sayısal bir değer tutması doğru değildir, çünkü hem üzerinde matematiksel işlem yapılmayacak hem de bazı zip kodları `0` ile başlayabilir.

In [ ]:
users["Zipcode"] = users["Zipcode"].astype(str)

_Gender_ sütunu tek bir cinsiyeti belirten birden fazla değere sahip olabilir, örneğin kadın için hem "F" hem de "Female" bulunabilir. Bunu kontrol etmeliyiz.

In [ ]:
users["Gender"].unique()

_Age_ sütunu gerçekçi bir aralık belirtmeli.

In [ ]:
users["Current Age"].value_counts().sort_index()

Eğer herhangi bir tekrarlama varsa bunlar kaldırılmalı.

In [ ]:
users.drop_duplicates(subset=["User"], inplace= True)

In [ ]:
# son hali
users.head()

In [ ]:
users.info()

Benzer işlemleri `transactions` veriseti için de yapmalıyız. 

In [ ]:
transactions.head()

In [ ]:
transactions.info()

`Merchant State` ve `Zip` sütunlarında boş değerler var. Dikkatli bakıldığında `Merchant State` değeri boş olan girdilerde `Merchant City` sütunu değerlerinin tamamının 'ONLINE' olduğunu görüyoruz. Buradan işlemlerin fiziksel bir mağazada değil online bir şekilde gerçekleştirildiğini çıkarabiliriz. Veri temizleme açısından `Merchant State`'i "Online" veya "Virtual" gibi bir kategori ile doldurmak daha anlamlı olabilir. Bu açıdan `Zip`değerlerini de 0 ile doldurup string veri tipi ile değiştirebiliriz.

In [ ]:
transactions[transactions["Merchant State"].isna()]

In [ ]:
transactions[transactions["Merchant State"].isna()]["Merchant City"].unique()

In [ ]:
transactions["Merchant State"] = transactions.apply(
	lambda x: "Online"
	if pd.isna(x["Merchant State"]) and x["Merchant City"] == "ONLINE"
	else x["Merchant State"],
	axis= 1
)

In [ ]:
transactions["Zip"] = transactions["Zip"].fillna(0).astype(int).astype(str)

In [ ]:
transactions[transactions["Merchant State"] == "Online"]

`Is Fraud?` sütunu binary bir sütun (Yes/No), bu değerleri 0/1 şeklinde sayısal veri olarak tutmak ileride modellemeyi kolaylaştırabilir.

In [ ]:
transactions["Is Fraud?"] = transactions["Is Fraud?"].map({"Yes": 1, "No": 0})

Veri setinde zamanı ifade etmek için 4 farklı sütun kullanılmış (`Year`, `Month`, `Day` ve `Time`), bu sütunları birleştirerek tek bir `datetime`kolonu oluşturmak zaman bazlı analizleri kolaylaştırabilir.

In [ ]:
transactions["Date"] = pd.to_datetime(
	transactions["Year"].astype(str) + "-" +
	transactions["Month"].astype(str) + "-" +
	transactions["Day"].astype(str) + " " +
	transactions["Time"]
)

transactions.drop(columns=["Year", "Month", "Day", "Time"], inplace= True)

In [ ]:
transactions.head()

`Merchant Name` sütunu her ne kadar sayısal bir değermiş gibi görünse de aslında kategoriktir. String'e çevirmek analiz ve gruplama işlemlerini kolaylaştırabilir.

In [ ]:
transactions["Merchant Name"] = transactions["Merchant Name"].astype(str)

`Errors?` sütununda çok fazla boş değer var fakat bu sütun ileride dolandırıcılık tahminlerinde kullanılabilecek önemli verileri içerebilir, bu sebeple `Is Fraud?` sütunundan "0" gelen satırlarda `Errors?` sütunlarına "No Error" yazmak doğru bir yaklaşım olabilir.

In [ ]:
transactions.loc[(transactions["Is Fraud?"] == 0) &
				 (transactions["Errors?"].isna()), "Errors?"] = "No Error"

`Is Fraud?` sütunundan "1" gelen satırlarda `Errors?` sütunlarına "Unknown Error" gibi bir etiket atmak daha güvenli olur.

In [ ]:
transactions.loc[(transactions["Is Fraud?"] == 1) &
				 (transactions["Errors?"].isna()), "Errors?"] = "Unknown Error"

Tekrarlı veri varsa onları da kaldırmalıyız.

In [ ]:
transactions.drop_duplicates(inplace= True)

In [ ]:
# son hali
transactions.head()

In [ ]:
transactions.info()

### Veri Setlerinin Birleştirilmesi

In [ ]:
merged_df = transactions.merge(users, on= "User", how= "inner")
merged_df.to_csv("merged_df.csv", index= False)

In [ ]:
merged_df.head()

In [ ]:
merged_df.info()

### Yapılacak İşlemler

In [ ]:
merged_df.columns

***En Fazla Harcama Yapılan Şehirler***

1. `Merchant City` kolonuna göre toplam harcama miktarını hesaplayın.

2. En fazla harcama yapılan ilk 10 şehri sıralayarak bir bar grafik ile gösterin.

In [ ]:
# 1.
city_spending = merged_df.groupby("Merchant City")["Amount"].sum().sort_values(ascending= False)
city_spending.head(10)

In [ ]:
# 2.
top10_cities = city_spending.head(10) # ilk 10 şehir

plt.figure(figsize=(10, 6))
top10_cities.plot(kind= "bar", color= "green", edgecolor= "black")

plt.title("En Çok Harcama Yapılan 10 Şehir", fontsize= 14)
plt.xlabel("Şehir", fontsize= 12)
plt.ylabel("Toplam Harcama", fontsize= 12)
plt.xticks(rotation= 45, ha= "right")
plt.savefig("images/graphs/en_fazla_harcama_10.png")
plt.show()

**Sonuç**

En çok harcamanın yapıldığı şehir La Verne olurken, onu açık ara farkla Monterey Park ve Mira Loma takip ediyor. Dikkat çekici olan diğer bir nokta ise, Online işlemlerinin dördüncü sırada olması. Bu durum kullanıcıların fiziksel mağazalar dışında çevrim içi alışverile de yoğun şekilde yöneldiğini gösteriyor.

***Saatlik Harcama Dağılımı***

1. Her saate karşılık gelen toplam harcamayı hesaplayın.

2. Sonuçları bir çizgi grafik ile gösterin.

In [ ]:
# 1.
merged_df["Hour"] = merged_df["Date"].dt.hour
hourly_spending = merged_df.groupby("Hour")["Amount"].sum()
hourly_spending

In [ ]:
# 2.
plt.figure(figsize= (10, 6))
hourly_spending.plot(kind= "line", marker= "o", linestyle= "-", color= "blue")

plt.title("Saatlik Harcama Dağılımı", fontsize= 14)
plt.xlabel("Saat", fontsize= 12)
plt.ylabel("Toplam Harcama", fontsize= 12)
plt.xticks(range(0, 24))
plt.savefig("images/graphs/saatlik_harcama.png")
plt.show()

**Sonuç**

En büyük harcama yoğunluğu sabah 6.00’da gerçekleşmiş. Bu kadar yüksek bir sıçrama, ya çok büyük tekil işlemlerden ya da belirli bir kampanya/promosyon etkisinden kaynaklanıyor olabilir. Ayrıca öğle 13.00 civarında da dikkat çekici bir artış var, bu da öğle tatilinde yapılan alışverişlere işaret edebilir. Akşam saatlerinde ise özellikle 20.00’de tekrar belirgin bir harcama artışı gözleniyor.

Gece yarısı ve sabaha karşı (0.00–4.00 arası) harcamalar oldukça düşük seviyede kalmış; bu da alışverişlerin ağırlıklı olarak gündüz ve akşam saatlerinde yoğunlaştığını gösteriyor.

***Cinsiyete Göre Harcama***

1. Kullanıcıların cinsiyetine göre toplam harcama miktarlarını gruplandırın.

2. Bar grafik yardımıyla karşılaştırma yapın.

In [ ]:
# 1.
gender_spending = merged_df.groupby("Gender")["Amount"].sum().sort_values(ascending= False)
gender_spending

In [ ]:
#2.
plt.figure(figsize=(6,4))
gender_spending.plot(kind="bar", color=["red", "blue"])

plt.title("Cinsiyete Göre Toplam Harcama")
plt.xlabel("Cinsiyet")
plt.ylabel("Toplam Harcama ($)")
plt.xticks(rotation=0)
plt.savefig("images/graphs/cinsiyete_gore_harcama.png")
plt.show()

**Sonuç**

Kullanıcıların cinsiyetlerine göre toplam harcama miktarları oldukçayakın. Kadın kullanıcılar toplam harcamada erkeklerden biraz önde, ancak fark çok büyük değil. Bu, her iki cinsiyetin de harcama alışkanlıklarının genel olarak dengeli olduğunu gösteriyor.

***Gelire Göre Harcama***

1. `Yearly Income - Person` kolonundaki `$` işaretini kaldırarak sayıya çevirin. (bunu zaten yapmıştık yukarıda)

2. Yıllık gelir ile harcama miktarı arasındaki ilişkiyi dağılım grafiği (scatter plot) ile gösterin.

In [ ]:
# 2.
plt.figure(figsize= (10, 6))
plt.scatter(
	merged_df["Yearly Income - Person"],
	merged_df["Amount"], alpha= 0.3, s=10, color= "green"
)

plt.title("Yıllık Gelir vs Harcama Miktarı")
plt.xlabel("Yıllık Gelir ($)")
plt.ylabel("Harcama Miktarı ($)")
plt.show()

Grafik bu haliyle pek okunaklı değil, daha okunabilir hale getirmeyi deneyebiliriz.

In [ ]:
x = merged_df["Yearly Income - Person"]
y = merged_df["Amount"]

# log dönüşümü
log_x = np.log1p(x)

plt.figure(figsize=(8,6))
plt.scatter(log_x, y, alpha=0.3, s=10, color="green")

plt.title("Yıllık Gelir vs Harcama Miktarı (Log Scale)")
plt.xlabel("Yıllık Gelir (Log $)")
plt.ylabel("Harcama Miktarı (Log $)")
plt.savefig("images/graphs/yillik_gelir_vs_harcama.png")
plt.show()

**Sonuç**

Genel olarak gelir arttıkça harcama da artıyor fakat çok sıkışık, özellikle orta gelir aralığında çok fazla nokta var. Düşük ve yüksek gelirlerde çok fazla uç nokta var, log scale bunu biraz düzeltmeye çalıştık ama hala yoğunluk farkları mevcut. Trend çizgisi eklenirsegenel artış eğilimi rahatça görülebilir.

### Aşama 3: Değerlendirme

ChatGPT'nin örneği (Python): Yaşa Göre Ortalama Harcama

In [ ]:
# Yaş gruplarını oluştur
bins = [18, 25, 35, 45, 55, 65, 75, 85, 100]
labels = ["18-24","25-34","35-44","45-54","55-64","65-74","75-84","85+"]
merged_df["Age Group"] = pd.cut(merged_df["Current Age"], bins=bins, labels=labels, right=False)

# Yaş grubuna göre ortalama harcama
age_spending = merged_df.groupby("Age Group")["Amount"].mean().reset_index()

# Bar chart
plt.figure(figsize=(10,6))
sns.barplot(x="Age Group", y="Amount", data=age_spending ,palette="viridis")
plt.title("Yaş Gruplarına Göre Ortalama Harcama")
plt.xlabel("Yaş Grubu")
plt.ylabel("Ortalama Harcama (USD)")
plt.savefig("images/graphs/chatgpt_ornegi.png")
plt.show()